In [5]:
import pandas as pd
import numpy as np

# Function to aggregate data into 15-minute intervals
def aggregate_data(file_path, estrus_days=None):
    df = pd.read_csv(file_path)

    # Determine the number of minutes per quarter-hour
    quarter_hour_bins = np.arange(0, 1441, 15)
    hourly_labels = np.arange(0, 24, 0.25)

    # Extract and reshape data
    reshaped_data = df.to_numpy().reshape(-1, 1440)  # Each row corresponds to one day

    # Compute averages for each quarter-hour bin
    avg_data = np.array([reshaped_data[:, i:i+15].mean(axis=1) for i in quarter_hour_bins[:-1]]).T

    # Convert to DataFrame
    avg_df = pd.DataFrame(avg_data, columns=hourly_labels)

    # If handling female data, split into estrus and non-estrus
    if estrus_days is not None:
        estrus_df = avg_df.iloc[estrus_days, :].mean(axis=0)
        non_estrus_df = avg_df.drop(index=estrus_days).mean(axis=0)
        return estrus_df, non_estrus_df

    return avg_df.mean(axis=0)  # Return overall averages for male data

# Estrus cycle days (Day 2, 6, 10, etc.)
estrus_days = np.concatenate([np.arange(2, 14, 4)])

# Process and save female data (estrus and non-estrus)
fem_act_estrus, fem_act_non_estrus = aggregate_data("FemAct.csv", estrus_days)
fem_temp_estrus, fem_temp_non_estrus = aggregate_data("FemTemp.csv", estrus_days)

fem_act_estrus.to_csv("AvgFemActEst.csv", index_label="hour")
fem_act_non_estrus.to_csv("AvgFemActNonEst.csv", index_label="hour")
fem_temp_estrus.to_csv("AvgFemTempEst.csv", index_label="hour")
fem_temp_non_estrus.to_csv("AvgFemTempNonEst.csv", index_label="hour")

# Process and save male data (no estrus cycle)
male_act = aggregate_data("MaleAct.csv")
male_temp = aggregate_data("MaleTemp.csv")

male_act.to_csv("AvgMaleAct.csv", index_label="hour")
male_temp.to_csv("AvgMaleTemp.csv", index_label="hour")



In [7]:
import pandas as pd
import numpy as np

# Load activity data
fem_act = pd.read_csv('FemAct.csv')
mal_act = pd.read_csv('MaleAct.csv')

# Melt data into long format
fem_act = fem_act.melt(var_name="Mouse", value_name="Activity")
mal_act = mal_act.melt(var_name="Mouse", value_name="Activity")

# Add Minute, Day, and Time_of_Day columns
fem_act["Minute"] = np.tile(np.arange(1440 * 14), len(fem_act["Mouse"].unique()))
fem_act["Day"] = fem_act["Minute"] // 1440
fem_act["Time_of_Day"] = fem_act["Minute"] % 1440  # Get minute within each day

mal_act["Minute"] = fem_act["Minute"]
mal_act["Day"] = fem_act["Day"]
mal_act["Time_of_Day"] = fem_act["Time_of_Day"]

# Define Night (Lights Off: 0-720 min) and Day (Lights On: 720-1440 min)
fem_act["Period"] = np.where(fem_act["Time_of_Day"] < 720, "Night", "Day")
mal_act["Period"] = fem_act["Period"]

In [2]:
import pandas as pd
import numpy as np

# Define constants
MINUTES_PER_DAY = 1440
INTERVAL = 15  # minutes per aggregation
HOURS = np.arange(0, 24, INTERVAL / 60)  # 0, 0.25, 0.5, ..., 23.75

# Estrus days pattern for females: starts on day 2, repeats every 4 days
estrus_days = set(range(2, 15, 4))  # Days: 2, 6, 10, 14

# Function to process and aggregate data
def process_file(filename, estrus_filter=None):
    df = pd.read_csv(filename)
    
    # Reshape to (minutes, mice) and compute 15-minute averages
    reshaped = df.to_numpy().reshape(14, MINUTES_PER_DAY, -1)
    
    # Compute quarter-hourly averages for each day
    qhour_averages = reshaped.reshape(14, 96, INTERVAL, -1).mean(axis=2)
    
    # Separate estrus and non-estrus days for females
    if estrus_filter is not None:
        estrus_data = qhour_averages[np.array([i+1 in estrus_days for i in range(14)])]
        non_estrus_data = qhour_averages[np.array([i+1 not in estrus_days for i in range(14)])]
        
        estrus_avg = estrus_data.mean(axis=(0, 2))
        non_estrus_avg = non_estrus_data.mean(axis=(0, 2))
        
        estrus_df = pd.DataFrame({'hour': HOURS, 'avg_value': estrus_avg})
        non_estrus_df = pd.DataFrame({'hour': HOURS, 'avg_value': non_estrus_avg})
        
        return estrus_df, non_estrus_df
    
    # For male data, just compute the overall average per interval
    avg_values = qhour_averages.mean(axis=(0, 2))
    return pd.DataFrame({'hour': HOURS, 'avg_value': avg_values})

# Process all files
fem_act_estr, fem_act_non_estr = process_file('FemAct.csv', estrus_filter=True)
fem_temp_estr, fem_temp_non_estr = process_file('FemTemp.csv', estrus_filter=True)
male_act = process_file('MaleAct.csv')
male_temp = process_file('MaleTemp.csv')

male_act_combined = pd.DataFrame({'hour': HOURS, 'avg_value': male_act['avg_value']})
male_temp_combined = pd.DataFrame({'hour': HOURS, 'avg_value': male_temp['avg_value']})

# Save to CSV
fem_act_estr.to_csv('AvgFemActEst.csv', index=False)
fem_act_non_estr.to_csv('AvgFemActNonEst.csv', index=False)
fem_temp_estr.to_csv('AvgFemTempEst.csv', index=False)
fem_temp_non_estr.to_csv('AvgFemTempNonEst.csv', index=False)
male_act.to_csv('AvgMaleActEst.csv', index=False)
male_act.to_csv('AvgMaleActNonEst.csv', index=False)
male_temp.to_csv('AvgMaleTempEst.csv', index=False)
male_temp.to_csv('AvgMaleTempNonEst.csv', index=False)
male_act_combined.to_csv('AvgMaleAct.csv', index=False)
male_temp_combined.to_csv('AvgMaleTemp.csv', index=False)
